In [2]:
import pandas
import random as rd

from flask import Flask, render_template, request, jsonify, send_from_directory
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
import os
import tempfile
import sqlite3
from pydub import AudioSegment

app = Flask(__name__)
translator = Translator()
recognizer = sr.Recognizer()

UPLOAD_FOLDER = os.path.join(app.root_path, 'static', 'audio')
TEMP_FOLDER = os.path.join(UPLOAD_FOLDER, 'temp_files')
os.makedirs(TEMP_FOLDER, exist_ok=True)
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

LANGUAGES = {
    "English": "en", "Spanish": "es", "French": "fr",
    "German": "de", "Hindi": "hi", "Chinese (Simplified)": "zh-cn",
    "Japanese": "ja"
}

def init_db():
    conn = sqlite3.connect("translations.db")
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS translations (
                          id INTEGER PRIMARY KEY AUTOINCREMENT,
                          recognized_text TEXT,
                          translated_text TEXT,
                          audio_file_path TEXT)''')
    conn.commit()
    conn.close()

def insert_translation(recognized_text, translated_text, audio_file_path):
    conn = sqlite3.connect("translations.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO translations (recognized_text, translated_text, audio_file_path) VALUES (?, ?, ?)",
                   (recognized_text, translated_text, audio_file_path))
    conn.commit()
    conn.close()

def convert_to_wav(input_file_path, output_file_path):
    audio = AudioSegment.from_file(input_file_path)
    audio.export(output_file_path, format="wav")
    return output_file_path

def speech_to_text(audio_file_path):
    with sr.AudioFile(audio_file_path) as source:
        audio = recognizer.record(source)
        return recognizer.recognize_google(audio)

@app.route('/')
def index():
    return render_template('index.html', languages=LANGUAGES)

@app.route('/translate', methods=['POST'])
def translate():
    try:
        if 'audio' not in request.files or 'language' not in request.form:
            return jsonify({'error': "Missing 'audio' file or 'language' parameter"}), 400

        audio_data = request.files['audio']
        target_lang = request.form['language']

        if target_lang not in LANGUAGES.values():
            return jsonify({'error': f"Unsupported language code: {target_lang}"}), 400

        with tempfile.NamedTemporaryFile(delete=False, dir=TEMP_FOLDER, suffix=".wav") as temp_audio:
            audio_data.save(temp_audio.name)
            input_audio_path = temp_audio.name

        with tempfile.NamedTemporaryFile(delete=False, dir=TEMP_FOLDER, suffix=".wav") as temp_converted:
            converted_audio_path = convert_to_wav(input_audio_path, temp_converted.name)

        recognized_text = speech_to_text(converted_audio_path)
        translated = translator.translate(recognized_text, dest=target_lang).text

        with tempfile.NamedTemporaryFile(delete=False, dir=UPLOAD_FOLDER, suffix=".mp3") as temp_translated_audio:
            tts = gTTS(translated, lang=target_lang)
            tts.save(temp_translated_audio.name)
            translated_audio_path = temp_translated_audio.name

        insert_translation(recognized_text, translated, os.path.basename(translated_audio_path))
        os.remove(input_audio_path)
        os.remove(converted_audio_path)

        return jsonify({
            'recognized_text': recognized_text,
            'translated_text': translated,
            'audio_file': f'/static/audio/{os.path.basename(translated_audio_path)}'
        })

    except Exception as e:
        return jsonify({'error': f"An unexpected error occurred: {str(e)}"}), 500

if __name__ == '__main__':
    init_db()
    app.run(debug=True)


c:\Users\arpit\PycharmProjects\Project_SIH\.venv\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

c:\Users\arpit\PycharmProjects\Project_SIH\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
%pip install gtts
from flask import Flask, render_template, request, jsonify
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
import sqlite3
import os

app = Flask(__name__)

# Initialize the translator and speech recognizer
translator = Translator()
recognizer = sr.Recognizer()

# Expanded list of languages including Indian languages and their codes for the dropdown menu
LANGUAGES = {
    "English": "en",
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Italian": "it",
    "Chinese (Simplified)": "zh-cn",
    "Japanese": "ja",
    "Korean": "ko",
    "Hindi": "hi",
    "Bengali": "bn",
    "Gujarati": "gu",
    "Kannada": "kn",
    "Malayalam": "ml",
    "Marathi": "mr",
    "Punjabi": "pa",
    "Tamil": "ta",
    "Telugu": "te",
    "Urdu": "ur",
}

# Initialize SQLite database
def init_db():
    conn = sqlite3.connect("translations.db")
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS translations
                      (id INTEGER PRIMARY KEY AUTOINCREMENT,
                      recognized_text TEXT,
                      translated_text TEXT,
                      audio_file_path TEXT)''')
    conn.commit()
    conn.close()

def insert_translation(recognized_text, translated_text, audio_file_path):
    conn = sqlite3.connect("translations.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO translations (recognized_text, translated_text, audio_file_path) VALUES (?, ?, ?)",
                   (recognized_text, translated_text, audio_file_path))
    conn.commit()
    conn.close()

# Flask Routes
@app.route('/')
def index():
    return render_template('template/index.html', languages=LANGUAGES)

@app.route('/translate', methods=['POST'])
def translate_audio():
    try:
        # Check if the audio file is provided
        if 'audio' not in request.files:
            return jsonify({'error': 'No audio file provided'}), 400

        # Save the audio file
        audio_file = request.files['audio']
        audio_file_path = os.path.join('uploads', audio_file.filename)
        os.makedirs('uploads', exist_ok=True)
        audio_file.save(audio_file_path)

        # Recognize speech
        with sr.AudioFile(audio_file_path) as source:
            recognizer.adjust_for_ambient_noise(source)
            audio = recognizer.record(source)
        input_text = recognizer.recognize_google(audio)

        # Translate the recognized text
        target_language = request.form.get('language')
        if target_language not in LANGUAGES.values():
            return jsonify({'error': 'Invalid target language selected'}), 400

        translated_text = translator.translate(input_text, dest=target_language).text

        # Convert the translated text to speech
        tts = gTTS(text=translated_text, lang=target_language)
        output_file_path = os.path.join('outputs', 'translated_speech.mp3')
        os.makedirs('outputs', exist_ok=True)
        tts.save(output_file_path)

        # Save the result to the database
        insert_translation(input_text, translated_text, output_file_path)

        return jsonify({
            'input_text': input_text,
            'translated_text': translated_text,
            'audio_file': output_file_path
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Initialize the database
init_db()

if __name__ == '__main__':
    app.run(debug=True)



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1